# Privacy-Preserving CNN

## Preparations

### options

In [1]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### modules

In [2]:
import torch  # pytorch
import numpy as np  # handling vectors and matrices
from keras.models import load_model  # reload pretrained tf models
from analysis_functions import (train_tf_model, pred_tf_model,
                                train_pt_model, pred_pt_model)  # own functions
from preproc_functions import list_combine  # own functions
from nn_arguments import Arguments  # nn options
from neural_nets import TFUNET, TFMNET, PTUNET, PTMNET  # own defined neural nets as classes

# pysyft
import syft as sy
from syft.frameworks.torch.tensors.interpreters import PointerTensor
from syft.frameworks.torch.tensors.decorators import LoggingTensor
hook = sy.TorchHook(torch)  # hooking pysyft into pytorch
import torch.nn.functional as F
import torch.nn as nn

Using TensorFlow backend.


### datasets

#### univariate training

In [3]:
train_Xu = np.load(dir_data+'train_Xu.npy')
train_yu = np.load(dir_data+'train_yu.npy')

In [4]:
print(train_Xu.shape)
print(train_yu.shape)

(988, 14, 1)
(988, 7)


995 Observations for X days for 1 Variable!

#### multivariate training

In [5]:
train_Xm = np.load(dir_data+'train_Xm.npy')
train_ym = np.load(dir_data+'train_ym.npy')

In [6]:
print(train_Xm.shape)
print(train_ym.shape)

(988, 14, 8)
(988, 7)


995 Observations for X days for 8 Variables!

#### test set

In [7]:
test_Xu = np.load(dir_data+'test_Xs.npy')[:,:,:1]
test_Xm = np.load(dir_data+'test_Xs.npy')
test_y = np.load(dir_data+'test_ys.npy')

In [8]:
print(test_Xu.shape)
print(test_Xm.shape)
print(test_y.shape)

(59, 14, 1)
(59, 14, 8)
(59, 7)


## Models

### Options

In [10]:
args = Arguments()
# overwrite timesteps depending on data
args.n_timesteps = train_Xu.shape[1]

### load pretrained weights

#### tensorflow

In [16]:
# create net objects
model_tfu = TFUNET(args.n_features_u, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)
model_tfm = TFMNET(args.n_features_m, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
# load weights
model_tfu.load_weights(dir_data+'models/weights_tfu.h5')
model_tfm.load_weights(dir_data+'models/weights_tfm.h5')

#### pytorch

In [18]:
# create net objects
model_ptu = PTUNET(args.n_features_u, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)
model_ptm = PTMNET(args.n_features_m, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)

In [19]:
# load weights
model_ptu.load_state_dict(torch.load(dir_data+'models/weights_ptu.pt'), strict=False)
model_ptm.load_state_dict(torch.load(dir_data+'models/weights_ptm.pt'), strict=False)

## Distribute Test data and model (all encrypted)

In [ ]:
# different parties
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

SMPC works with integers so we need to convert floats to int via fix_precision.

In [ ]:
torch.tensor(test_Xu).fix_precision().share(alice, bob, crypto_provider=crypto_provider)
torch.tensor(test_y).fix_precision().share(alice, bob, crypto_provider=crypto_provider)

In [ ]:
model_ptu.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

In [ ]:
def test(model, X, y):
    model.eval()
    with torch.no_grad():
        output = model(X)
        pred = output.argmax(dim=1) 
        n_correct_priv += pred.eq(y.view_as(pred)).sum()
        n_total += args.test_batch_size